<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/python/P03S01_Classification-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification-1

In [1]:
!git clone https://github.com/saiku122/AIJobcolle.git

Cloning into 'AIJobcolle'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 109 (delta 37), reused 54 (delta 12), pack-reused 0
Receiving objects: 100% (109/109), 2.53 MiB | 12.29 MiB/s, done.
Resolving deltas: 100% (37/37), done.


2値分類用のサンプルデータである乳がんデータを読み込みます。<br>特徴量が全て数値データで欠損値もないキレイなサンプルデータです。

In [2]:
cd /content/AIJobcolle/MachineLearning/python

/content/AIJobcolle/MachineLearning/python


In [3]:
# Load and return the breast cancer wisconsin dataset (classification).
# The breast cancer dataset is a classic and very easy binary classification dataset.
import pandas as pd
from sklearn.datasets import load_breast_cancer

# set data by role
dataset = load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.Series(dataset.target, name='y')

# check the shape
print('--------------------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('--------------------------------------------------')
print(y.value_counts())
print('--------------------------------------------------')
print('y=0 means Marignant(悪性),y=1 means Benign(良性):')
print('--------------------------------------------------')
X.join(y).head()

--------------------------------------------------
X shape: (569,30)
--------------------------------------------------
1    357
0    212
Name: y, dtype: int64
--------------------------------------------------
y=0 means Marignant(悪性),y=1 means Benign(良性):
--------------------------------------------------


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,y
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


次に評価指標についてです。回帰モデルの評価にはR2スコアなどがありました。分類モデルの評価の場合、最もわかりやすいものが正解率（Accuracy）です。正解率は予測対象データ（スコアリングデータ）の全件をNとした時、正例を正しく正例と分類できた数TP、負例を正しく負例と分類できた数TFとすると、 以下の数式で表せます。<br><center>Accuracy = (TP + TF) / N</center><br>つまり、正例・負例を問わず正しく分類されたサンプル数の割合です。sklearnでは正解データと予測データを与えると、Accuracyを計算してくれる関数が存在します。以下で呼び出しておきます。

In [4]:
# 分類モデルの評価指標計算のための関数の読込
from sklearn.metrics import accuracy_score

下記コードは回帰と同様ですので、各行の役割を理解できていることを確認して下さい。

In [7]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Holdout
X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 test_size=0.20,
                                                 random_state=1)
# set pipelines for two different algorithms
pipelines ={
    'knn': Pipeline([('scl',StandardScaler()),
                       ('est',KNeighborsClassifier())]),

    'logistic': Pipeline([('scl',StandardScaler()),
                          ('est',LogisticRegression(random_state=1))])    
}

# fit the models
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_train,y_train)
    print(pipe_name, ': Fitting Done')

knn : Fitting Done
logistic : Fitting Done


In [14]:
from sklearn.metrics import f1_score
print(f1_score(y_test.values.ravel(),pipelines["knn"].predict(X_test))) #k近傍
print(f1_score(y_test.values.ravel(),pipelines["logistic"].predict(X_test))) #logistic回帰

0.9664429530201343
0.9793103448275863


学習を終えたので、２つのパイプラインの分類器としての性能（正解率）を比較します。<br>まず、pipelineのpredictメソッドによる出力ベクトルを見て下さい。<br>y_pred in {0,1}で予測ラベルが付与されていることがわかります。

In [15]:
pipelines['logistic'].predict(X_train)[:10]

array([0, 0, 1, 1, 1, 1, 1, 0, 0, 1])

ちなみに、今回は使いませんが、実務では予測ラベルの他に各クラスに分類される予測確率をよく使います。<b>予測確率を出力させたい場合はpredictをpredic_probaに変更します。</b>最初の列はy=0と分類される予測確率、2番目の列はy=1と分類される予測確率です。

In [16]:
pipelines['logistic'].predict_proba(X_train)[:10]

array([[9.99418236e-01, 5.81764051e-04],
       [9.99957753e-01, 4.22467006e-05],
       [1.05256053e-06, 9.99998947e-01],
       [4.02739137e-04, 9.99597261e-01],
       [1.75511145e-04, 9.99824489e-01],
       [7.77824273e-03, 9.92221757e-01],
       [6.66717409e-02, 9.33328259e-01],
       [9.99953387e-01, 4.66128892e-05],
       [9.99999129e-01, 8.70634846e-07],
       [2.29495984e-02, 9.77050402e-01]])

さて、accuracy_scoreは第一引数に正解ラベル、第二引数に予測ラベル（確率ではない）を指定します。ですので、今回はpredictメソッドで予測ラベルを与えましょう。

In [17]:
scores = {}
for pipe_name, pipeline in pipelines.items():
    scores[(pipe_name,'train')] = accuracy_score(y_train, pipeline.predict(X_train))
    scores[(pipe_name,'test')] = accuracy_score(y_test, pipeline.predict(X_test))

pd.Series(scores).unstack()

,test,train
knn,0.956140,0.982418
logistic,0.973684,0.991209


このデータに対しては、kNNよりもロジスティック回帰による精度（正解率）の方が勝ることを確認できました。アルゴリズムの知識を増やして、様々なパイプラインを設計できるようになりましょう。